In [ ]:
import os, numpy as np, csv
import pandas as pd
# Uncomment this if using python2
#from __future__ import division # Used with Jupyter (python2) to make int -> f division work


######################
# SEE BELOW COMMENTS #
######################

# CHANGE THIS 
#Directories, !!! Make sure that you use "/" sclashes, not "\", Python does not like it. And finish all directories with /
#Example: 'C:/Users/31380/final/'
converter_Dir='/home/student/Downloads/IS_Project-master/'
input_Dir='/home/student/Downloads/IS_Project-master/input/'
output_Dir='/home/student/Downloads/IS_Project-master/output/'

#Settings 'Relative' or 'Absolute' time
time_Midi='Relative'

#MIDI settings, each channel 1-16 can have individual music instruments, can be found here: https://en.wikipedia.org/wiki/General_MIDI
array_Instrument=[5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5]
nTempo = 333

# USER CAN SET THIS, changes amount of notes used to predict
windowLen = 4

# USER CAN CHANGE THESE, changes learning properties of the neural networks
learningRate = 0.001
learningMomentum = 0.8
noIter =  500
unitsNote = 5
unitsDur = 3
unitsVol = 5


#print('Converter:',converter_Dir+'midicsv.exe')
#print('Input Midi files:'+input_Dir)
#print('Output CSV files:'+output_Dir)

list_Midi=[0 for i in range(len(os.listdir(input_Dir)))]
q=0

for file in os.listdir(input_Dir):
    if file.endswith(".mid"):
        list_Midi[q]=file
        q=q+1
        
list_Midi=np.trim_zeros(list_Midi)
print(list_Midi)
if len(list_Midi)==0:
    print('Input directory is empty')

for the_file in os.listdir(output_Dir):
    file_path = os.path.join(output_Dir, the_file)
    try:
        if os.path.isfile(file_path) and the_file.endswith(".csv"):
            os.unlink(file_path)
    except Exception as e:
        print(e)
    
#####################################################
# IF YOU ARE RUNNING IN WINDOWS REMOVE 'wine '+ HERE#
#####################################################

for i in range(0,len(list_Midi)):
    state=os.system('wine '+converter_Dir+'midicsv.exe '+input_Dir+list_Midi[i]+'>'+output_Dir+list_Midi[i].replace(".mid",".csv"))
    if state==1:
        print('Error: file '+list_Midi[i]+'convertion problem.')
        state=0
        
list_CSV=[0 for i in range(len(os.listdir(output_Dir)))]
q=0
for file in os.listdir(output_Dir):
    if file.endswith(".csv"):
        list_CSV[q]=file
        q=q+1
list_CSV=np.trim_zeros(list_CSV)

for i in range(0,len(list_CSV)):
    with open(output_Dir+list_CSV[i], 'r') as inp, open(output_Dir+list_CSV[i].replace(".csv","_e.csv"), 'w') as out:
        writer = csv.writer(out)
        for row in csv.reader(inp):
            if row[2] == " Note_on_c" or row[2] == " Note_off_c":
                writer.writerow(row)

q=0                
for i in range(0,len(list_CSV)):
    data=pd.read_csv(output_Dir+list_CSV[i].replace(".csv","_e.csv"),
                     names=['Channel','Time','State','Something','Note','Volume'],
                     index_col = False,  error_bad_lines=False, header=None, sep=',')
    data=data[['Channel','Time','State','Note','Volume']]
    if time_Midi=='Relative':
        for n in range(0,len(data)):
            try:
                if data.ix[n,'State'] != ' Note_off_c' or (data.ix[n,'Volume'] != 0 and data.ix[n,'State'] == ' Note_on_c'):
                    data.ix[n,'Time']=data.ix[n+1,'Time']-data.ix[n,'Time']
            except Exception as e:
                e
        for n in range(0,len(data)):
            if data.ix[n,'State'] == ' Note_off_c' or (data.ix[n,'Volume'] == 0 and data.ix[n,'State'] == ' Note_on_c') or data.ix[n,'Time'] == 0:
                data=data.drop(n)
    data.to_csv(output_Dir+list_CSV[q].replace(".csv","_e.csv"), index = False)


    
# Neural network setup. 
from sklearn import cross_validation
from sknn.mlp import MultiLayerPerceptron, Classifier, Regressor, Layer

nn_Pitch = Regressor(layers=[Layer("Rectifier", units=unitsNote),Layer("Linear")],learning_rate=learningRate,learning_momentum=learningMomentum, debug=True,n_iter=noIter)
nn_Vol = Regressor(layers=[Layer("Rectifier", units=unitsDur),Layer("Linear")],learning_rate=learningRate,learning_momentum=learningMomentum, debug=True,n_iter=noIter)
nn_Dur = Regressor(layers=[Layer("Rectifier", units=unitsVol),Layer("Linear")],learning_rate=learningRate,learning_momentum=learningMomentum,debug=True,n_iter=noIter)

eCSV = pd.read_csv(output_Dir+list_CSV[q].replace(".csv","_e.csv"))
#print(eCSV)

inputChannel=[0 for y in range(16)]
inputSongLen=[0 for y in range(16)]

for x in range(16):
    inputChannel[x] = eCSV.loc[eCSV['Channel'] == x+1]
    inputSongLen[x] = len(inputChannel[x])
#print(inputSongLen)
#print(len(inputChannel))


noteTest = [[0 for z in range(windowLen)],[0 for z in range(windowLen)]]
durTest = [[0 for z in range(windowLen)],[0 for z in range(windowLen)]]
volTest = [[0 for z in range(windowLen)],[0 for z in range(windowLen)]]
NNoutput = [0 for z in range(16)]

for i in range(len(inputChannel)):
    if inputChannel[i].empty == False: 
        if len(inputChannel[i])>2*windowLen:     
            # Array setup
            y_train_note=[0 for y in range(1)]
            y_train_vol=[0 for y in range(1)]
            y_train_dur=[0 for y in range(1)]
            Note_train2=[0 for y in range(windowLen)]
            Note_train3=[0 for y in range(windowLen)]
            Vol_train2=[0 for y in range(windowLen)]
            Vol_train3=[0 for y in range(windowLen)]
            Dur_train2=[0 for y in range(windowLen)]
            Dur_train3=[0 for y in range(windowLen)]
            note_test=[0 for z in range(len(inputChannel[i]))]
            dur_test=[0 for z in range(len(inputChannel[i]))]
            vol_test=[0 for z in range(len(inputChannel[i]))]
            channel = [i+1 for z in range(len(inputChannel[i]))]
            State = ['Note_on_c' for z in range(len(inputChannel[i]))]
            Time = [0 for z in range(len(inputChannel[i]))]
            

            # Import channel data
            #for g in len(inputChannel[i]):
            Note_train = inputChannel[i].Note
            Vol_train = inputChannel[i].Volume
            Dur_train = inputChannel[i].Time

           #print('Channel', i+1)

            # Train on one channel at a time    
            for y in range(inputChannel[i].axes[0][0],inputChannel[i].axes[0][len(inputChannel[i])-1]-windowLen):
                for x in range(windowLen):
                    Note_train2[x]=Note_train[x+y]/128
                    Vol_train2[x]=Vol_train[x+y]/100
                    Dur_train2[x]=Dur_train[x+y]/4096   
                if y == 0:
                    Note_train3=Note_train2
                    Vol_train3=Vol_train2
                    Dur_train3=Dur_train2
                    y_train_note[0]=Note_train[inputChannel[i].axes[0][0]+windowLen]/128
                    y_train_vol[0]=Vol_train[inputChannel[i].axes[0][0]+windowLen]/128
                    y_train_dur[0]=Dur_train[inputChannel[i].axes[0][0]+windowLen]/4096
                else:
                    Note_train3=np.vstack((Note_train3, Note_train2))
                    Vol_train3=np.vstack((Vol_train3, Vol_train2))
                    Dur_train3=np.vstack((Dur_train3, Dur_train2))
                    y_train_note=np.vstack((y_train_note, Note_train[y+windowLen]/128))
                    y_train_vol=np.vstack((y_train_vol, Vol_train[y+windowLen]/128))
                    y_train_dur=np.vstack((y_train_dur, Dur_train[y+windowLen]/4096))

            # Fit NN with data from one channel    
            nn_Pitch.fit(Note_train3, y_train_note)
            nn_Vol.fit(Vol_train3, y_train_vol)
            nn_Dur.fit(Dur_train3, y_train_dur)

            # Generates notes for length of input channel. Might have to edit depending on how many songs we import 
            for y in range(len(inputChannel[i])-2*windowLen):
                #print 'iteration', y
                if y == 0:
                    # Create initial data
                    z=0
                    for x in range(inputChannel[i].axes[0][0]+y, inputChannel[i].axes[0][0]+y+2*windowLen):
                        note_test[z]=Note_train[x]/128
                        dur_test[z]=Dur_train[x]/4096
                        vol_test[z]=Vol_train[x]/128
                        z=z+1
                    noteTest[0] = note_test[0:windowLen]
                    noteTest[1] = note_test[windowLen:2*windowLen]

                    # Actually generate notes    
                    note_test[2*windowLen]=nn_Pitch.predict(np.asarray(noteTest))[1,0]
                    dur_test[2*windowLen]=nn_Dur.predict(np.asarray(durTest))[1,0]
                    vol_test[2*windowLen]=nn_Vol.predict(np.asarray(volTest))[1,0]

                else:
                    # Takes previously generated notes and puts them back into the NN
                    noteTest[0] = note_test[y:windowLen+y]
                    noteTest[1] = note_test[windowLen+y:2*windowLen+y]

                    note_test[2*windowLen+y]=nn_Pitch.predict(np.asarray(noteTest))[1,0]
                    dur_test[2*windowLen+y]=nn_Dur.predict(np.asarray(durTest))[1,0]
                    vol_test[2*windowLen+y]=nn_Vol.predict(np.asarray(volTest))[1,0]
                    
            #########################################################################
            # uncomment these if recieving garbage (read: negative) values in output#
            #########################################################################
            
            #vol_test = [vol_test[1] for z in range(len(inputChannel[i]))]
            #dur_test = [dur_test[1] for z in range(len(inputChannel[i]))]
            npoutput = [np.squeeze(channel).tolist(),np.squeeze(np.round(np.asarray(dur_test)*4096)).tolist(),np.squeeze(State).tolist(),np.squeeze(np.round(np.asarray(note_test)*128)).tolist(),np.squeeze(np.round(np.asarray(vol_test)*100)).tolist(),np.squeeze(Time).tolist()]
            
            NNoutput[i] = npoutput
            
#print(NNoutput)


output = [[0 for y in range(len(eCSV))],[0 for y in range(len(eCSV))],[0 for y in range(len(eCSV))],[0 for y in range(len(eCSV))],[0 for y in range(len(eCSV))],[0 for y in range(len(eCSV))]] 

# Check bit, since i == 0 is not necessarily the first channel used.
x=0 

for i in range(16):
    # Only convert long enough outputs
    if len(inputChannel[i])>2*windowLen: 
        if x == 0:
            chOut = NNoutput[i][0] 
            durOut = NNoutput[i][1]
            stateOut = NNoutput[i][2]            
            noteOut = NNoutput[i][3]
            volOut = NNoutput[i][4] 
            timeOut = NNoutput[i][5]
            # Increment check bit when first sucessful run is completed
            x+=1 
        else: 
            noteOut = np.vstack((np.asarray(noteOut).reshape(-1,1), np.asarray(NNoutput[i][3]).reshape(-1,1))) 
            durOut = np.vstack((np.asarray(durOut).reshape(-1,1), np.asarray(NNoutput[i][1]).reshape(-1,1))) 
            volOut = np.vstack((np.asarray(volOut).reshape(-1,1), np.asarray(NNoutput[i][4]).reshape(-1,1))) 
            timeOut = np.vstack((np.asarray(timeOut).reshape(-1,1), np.asarray(NNoutput[i][5]).reshape(-1,1))) 
            stateOut = np.vstack((np.asarray(stateOut).reshape(-1,1), np.asarray(NNoutput[i][2]).reshape(-1,1))) 
            chOut = np.vstack((np.asarray(chOut).reshape(-1,1), np.asarray(NNoutput[i][0]).reshape(-1,1))) 
            
# Reshape to correct dataframe format            
output[0] = chOut.reshape(-1,).tolist() 
output[1] = durOut.reshape(-1,).tolist() 
output[2] = stateOut.reshape(-1,).tolist() 
output[3] = noteOut.reshape(-1,).tolist() 
output[4] = volOut.reshape(-1,).tolist() 
output[5] = timeOut.reshape(-1,).tolist() 
output = pd.DataFrame(data=output, index=['Channel','Duration','State','Note','Volume','Time']).transpose() 



data_file = output
data_file=data_file.reset_index(drop=True)
data_M = data_file
data_file = data_file[['Channel','Time','State','Note','Volume']]
data_out = pd.DataFrame([],columns = ['Channel','Time','State','Note','Volume'])
chan=data_file.Channel[1]
Time=0

for i in range(len(data_file)):
    if data_file.Channel[i]-chan==1:
        Time=0
    
    chan=data_file.Channel[i]
    data_out = data_out.append(data_file[i:i+1])
    data_out.Time[i]=Time
    data_out = data_out.append(pd.DataFrame([[data_file.Channel[i], Time+data_M.Duration[i], ' Note_on_c',data_file.Note[i], 0]], columns = ['Channel','Time','State','Note','Volume']))
    data_M.Time[i+1] = data_M.Time[i]+data_M.Duration[i]
    Time=Time+data_M.Duration[i]
    
data_out=data_out.reset_index(drop=True)
    
data = data_out
array_Channel_1 = array_Channel_2 = array_Channel_3 = array_Channel_4 = array_Channel_5 = array_Channel_6 = pd.DataFrame([],columns = ['Channel','Time','State','Program','Note','Volume'])
array_Channel_7 = array_Channel_8 = array_Channel_9 = array_Channel_10 = array_Channel_11 = array_Channel_12 = pd.DataFrame([],columns = ['Channel','Time','State','Program','Note','Volume'])
array_Channel_13 = array_Channel_14 = array_Channel_15 = array_Channel_16 = pd.DataFrame([],columns = ['Channel','Time','State','Program','Note','Volume'])


try:
    data.insert(3,'Program',1)
except Exception as e:
    e

for i in range(0,len(data)):
    if data.Channel[i] == 1:
        array_Channel_1 = array_Channel_1.append(data[i:i+1])
        array_Channel_1['Program'] = 0 
    elif data.Channel[i] == 2:
        array_Channel_2 = array_Channel_2.append(data[i:i+1])
        array_Channel_2['Program'] = 1
    elif data.Channel[i] == 3:
        array_Channel_3 = array_Channel_3.append(data[i:i+1])
        array_Channel_3['Program'] = 2
    elif data.Channel[i] == 4:
        array_Channel_4 = array_Channel_4.append(data[i:i+1])
        array_Channel_4['Program'] = 3
    elif data.Channel[i] == 5:
        array_Channel_5 = array_Channel_5.append(data[i:i+1])
        array_Channel_5['Program'] = 4
    elif data.Channel[i] == 6:
        array_Channel_6 = array_Channel_6.append(data[i:i+1])
        array_Channel_6['Program'] = 5
    elif data.Channel[i] == 7:
        array_Channel_7 = array_Channel_7.append(data[i:i+1])
        array_Channel_7['Program'] = 6
    elif data.Channel[i] == 8:
        array_Channel_8 = array_Channel_8.append(data[i:i+1])
        array_Channel_8['Program'] = 7
    elif data.Channel[i] == 9:
        array_Channel_9 = array_Channel_9.append(data[i:i+1])
        array_Channel_9['Program'] = 8
    elif data.Channel[i] == 10:
        array_Channel_10 = array_Channel_10.append(data[i:i+1])
        array_Channel_10['Program'] = 9
    elif data.Channel[i] == 11:
        array_Channel_11 = array_Channel_11.append(data[i:i+1])
        array_Channel_11['Program'] = 10
    elif data.Channel[i] == 12:
        array_Channel_12 = array_Channel_12.append(data[i:i+1])
        array_Channel_12['Program'] = 11
    elif data.Channel[i] == 13:
        array_Channel_13 = array_Channel_13.append(data[i:i+1])
        array_Channel_13['Program'] = 12
    elif data.Channel[i] == 14:
        array_Channel_14 = array_Channel_14.append(data[i:i+1])
        array_Channel_14['Program'] = 13
    elif data.Channel[i] == 15:
        array_Channel_15 = array_Channel_15.append(data[i:i+1])
        array_Channel_15['Program'] = 14

channel_Leng = [array_Channel_1.empty, array_Channel_2.empty, array_Channel_3.empty, array_Channel_4.empty, 
                array_Channel_5.empty, array_Channel_6.empty, array_Channel_7.empty, array_Channel_8.empty, 
                array_Channel_8.empty, array_Channel_9.empty, array_Channel_10.empty, array_Channel_11.empty, 
                array_Channel_12.empty, array_Channel_13.empty, array_Channel_14.empty, array_Channel_15.empty, 
                array_Channel_16.empty]

settings_Midi = pd.DataFrame(columns = ['Channel','Time','State','Program','Note','Volume'])
for i in range(16):
    settings_Midi=settings_Midi.append(pd.DataFrame([[i, 0,' Start_track', 0, 0, 0],
                                                     [i, 0, 'MIDI_port',0],
                                                     [i, 0,' Program_c', i-1, array_Instrument[i], 0],
                                                     [i, 0,' Control_c', i-1, array_Instrument[i], 100],
                                                     [i, 0,' Control_c', i-1, array_Instrument[i], 0]],
                                                   columns = ['Channel','Time','State','Program','Note','Volume']))


data=data[['Channel','Time','State','Program','Note','Volume']]
settings_Midi_head = pd.DataFrame([[0, 0, 'Header', 1, max(data.Channel), 384],
                              [0, 0, 'Tempo', (nTempo*1000)]],
                            columns = ['Channel','Time','State','Program','Note','Volume'] )

settings_Midi_end = pd.DataFrame([[0, 0, 'End_of_file', 0, 0, 0]],
                            columns = ['Channel','Time','State','Program','Note','Volume'])



data=settings_Midi.append(data)
data=data.append(settings_Midi_end)

# Chanell 1 append

settings_Midi_head = settings_Midi_head.append(settings_Midi[5:10])
array_Channel_1 = settings_Midi_head.append(array_Channel_1)
if not channel_Leng[0]:
    array_Channel_1 = array_Channel_1.append(pd.DataFrame([[1,max(array_Channel_1.Time), ' End_track']],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_1 = array_Channel_1.append(pd.DataFrame([[1,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))

# Chanell 2 append
array_Channel_2 = settings_Midi[10:15].append(array_Channel_2)
if not channel_Leng[1]:
    array_Channel_2 = array_Channel_2.append(pd.DataFrame([[2,max(array_Channel_2.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_2 = array_Channel_2.append(pd.DataFrame([[2,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
# Chanell 3 append
array_Channel_3 = settings_Midi[15:20].append(array_Channel_3)
if not channel_Leng[2]:
    array_Channel_3 = array_Channel_3.append(pd.DataFrame([[3,max(array_Channel_3.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_3 = array_Channel_3.append(pd.DataFrame([[3,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
# Chanell 4 append
array_Channel_4 = settings_Midi[20:25].append(array_Channel_4)
if not channel_Leng[3]:
    array_Channel_4 = array_Channel_4.append(pd.DataFrame([[4,max(array_Channel_4.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_4 = array_Channel_4.append(pd.DataFrame([[4,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
# Chanell 5 append
array_Channel_5 = settings_Midi[25:30].append(array_Channel_5)
if not channel_Leng[4]:
    array_Channel_5 = array_Channel_5.append(pd.DataFrame([[5,max(array_Channel_5.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_5 = array_Channel_5.append(pd.DataFrame([[5,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
    
# Chanell 6 append
array_Channel_6 = settings_Midi[30:35].append(array_Channel_6)
if not channel_Leng[5]:
    array_Channel_6 = array_Channel_6.append(pd.DataFrame([[6,max(array_Channel_6.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_6 = array_Channel_6.append(pd.DataFrame([[6,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
# Chanell 7 append
array_Channel_7 = settings_Midi[35:40].append(array_Channel_7)
if not channel_Leng[6]:
    array_Channel_7 = array_Channel_7.append(pd.DataFrame([[7,max(array_Channel_7.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_7 = array_Channel_7.append(pd.DataFrame([[7,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 8 append
array_Channel_8 = settings_Midi[40:45].append(array_Channel_8)
if not channel_Leng[7]:
    array_Channel_8 = array_Channel_8.append(pd.DataFrame([[8,max(array_Channel_8.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_8 = array_Channel_8.append(pd.DataFrame([[8,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 
    
# Chanell 9 append
array_Channel_9 = settings_Midi[45:50].append(array_Channel_9)
if not channel_Leng[8]:
    array_Channel_9 = array_Channel_9.append(pd.DataFrame([[9,max(array_Channel_9.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_9 = array_Channel_9.append(pd.DataFrame([[9,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 10 append
array_Channel_10 = settings_Midi[50:55].append(array_Channel_10)
if not channel_Leng[9]:
    array_Channel_10 = array_Channel_10.append(pd.DataFrame([[10,max(array_Channel_10.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_10 = array_Channel_10.append(pd.DataFrame([[10,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 11 append
array_Channel_11 = settings_Midi[55:60].append(array_Channel_11)
if not channel_Leng[10]:
    array_Channel_11 = array_Channel_11.append(pd.DataFrame([[11,max(array_Channel_11.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_11 = array_Channel_11.append(pd.DataFrame([[11,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 12 append
array_Channel_12 = settings_Midi[60:65].append(array_Channel_12)
if not channel_Leng[11]:
    array_Channel_12 = array_Channel_12.append(pd.DataFrame([[12,max(array_Channel_12.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_12 = array_Channel_12.append(pd.DataFrame([[12,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 13 append
array_Channel_13 = settings_Midi[65:70].append(array_Channel_13)
if not channel_Leng[12]:
    array_Channel_13 = array_Channel_13.append(pd.DataFrame([[13,max(array_Channel_13.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_13 = array_Channel_13.append(pd.DataFrame([[13,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))

# Chanell 14 append
array_Channel_14 = settings_Midi[70:75].append(array_Channel_14)
if not channel_Leng[13]:
    array_Channel_14 = array_Channel_14.append(pd.DataFrame([[14,max(array_Channel_14.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_14 = array_Channel_14.append(pd.DataFrame([[14,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume'])) 

# Chanell 15 append
array_Channel_15 = settings_Midi[75:80].append(array_Channel_15)
if not channel_Leng[14]:
    array_Channel_15 = array_Channel_15.append(pd.DataFrame([[15,max(array_Channel_15.Time), ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))
else:
    array_Channel_15 = array_Channel_15.append(pd.DataFrame([[15,0, ' End_track',0,0,0]],
                                           columns = ['Channel','Time','State','Program','Note','Volume']))


    
data_out = pd.DataFrame([],columns = ['Channel','Time','State','Program','Note','Volume'])

data_out = data_out.append(array_Channel_1)
data_out = data_out.append(array_Channel_2)
data_out = data_out.append(array_Channel_3)
data_out = data_out.append(array_Channel_4)
data_out = data_out.append(array_Channel_5)
data_out = data_out.append(array_Channel_6)
data_out = data_out.append(array_Channel_7)
data_out = data_out.append(array_Channel_8)
data_out = data_out.append(array_Channel_9)
data_out = data_out.append(array_Channel_10)
data_out = data_out.append(array_Channel_11)
data_out = data_out.append(array_Channel_12)
data_out = data_out.append(array_Channel_13)
data_out = data_out.append(array_Channel_14)
data_out = data_out.append(array_Channel_15)
data_out = data_out.append(settings_Midi_end)

data_out['Channel']=data_out['Channel'].fillna(0)
data_out['Time']=data_out['Time'].fillna(0)
data_out['Program']=data_out['Program'].fillna(0)
data_out['Note']=data_out['Note'].fillna(0)
data_out['Volume']=data_out['Volume'].fillna(0)

data_out['Channel'] = data_out['Channel'].astype(int)
data_out['Time'] = data_out['Time'].astype(int)
data_out['Program'] = data_out['Program'].astype(int)
data_out['Note'] = data_out['Note'].astype(int)
data_out['Volume'] = data_out['Volume'].astype(int)

data_out=data_out[['Channel','Time','State','Program','Note','Volume']]
data_out.to_csv('Output.csv',index=False, header = False)

state=os.system('wine '+converter_Dir+'csvmidi.exe '+'Output.csv>Output.mid')

print('Done')
            